# 공간시각화 

- 위치정보(경도,위도)를 지도에 매핑하여 시각적으로 표현 하는 것.

- 단순 이미지가 아닌 인터랙티브한 활용이 가능하다.

- 거시적에서 미시적으로 진행되는 분석 방향과 같이 스토리라인을 잡고 시각화를 적용하는 것이 좋다.

> 도트맵 

- 위치에 동일한 크기의 작은 점을 찍어서 해당 지역의 데이터 분포나 패턴을 표현하는 기법.

- 데이터의 개요를 파악하는데 유리.

> 버블맵

- 버븥차트를 지도에 옮겨 둔 것이라고 생각 하면 됨.

- 데이터의 값이 크기로 표현 되기 때문에 도트맵에 비해서 크기를 비교 할 수 있음.

- 버블간 중첩에는 유의 ( 비율이 아닌 크기 )

> 코로플레스 맵

- 데이터 값의 크기에 따라서 색상의 음영을 달리하는 기법.

- 단순 지역의 크게에 따라서 강조 되는 인상을 가질 수 있다는 점에 대해서 유념.

> 커넥션 맵 혹은 링크 맵

- 지리적 관계를 표현 뿐만 아니라 연속적 연결을 통해서 경로를 표현 할 수 있다.

- 연결선의 분포와 집중도를 통해 지리적 관계의 패턴을 파악 하기 좋다.

> 그외

- 시작점과 도착점이 함께 표현 되는 **플로우 맵**

- 각 지역의 면적을 데이터 값에 비례하여 변형시키는 **카토그램**

In [1]:
# 필요한 패키지 설치 및 임포트

!pip install folium
import folium 
from folium import Marker
from folium import plugins
from folium import GeoJson
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
plt.rcParams['figure.dpi']=150

In [ ]:
# 데이터 불러오기

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/study_dataset/starbucks_seoul/Starbucks_Seoul.csv',low_memory=False)

# 지역 구분을 위한 Json 파일 불러오기

# geo = "/content/drive/MyDrive/Colab Notebooks/study_dataset/starbucks_seoul/Seoul_Gu.json"

df.head()

,name,address,gu_name,latitude,longitude
0,GS타워,서울특별시 강남구 논현로 508 (역삼동),강남구,37.501859,127.037278
1,SSG마켓도곡R,"서울특별시 강남구 언주로30길 57, 타워팰리스Ⅱ F 지하1층 (도곡동)",강남구,37.490298,127.054895
2,W-Mall,서울특별시 금천구 디지털로 188 (가산동),금천구,37.477305,126.887691
3,가든파이브,서울특별시 송파구 충민로 10 (문정동) 가든파이브툴,송파구,37.478232,127.119370
4,가락본동,서울특별시 송파구 송파대로30길 13 (가락동),송파구,37.494895,127.118785


In [8]:
import chardet

with open("/content/drive/MyDrive/Colab Notebooks/study_dataset/starbucks_seoul/Seoul_Gu.json") as f:

  data = f.readline()
  
print(chardet.detect(data))

# import json
# json.loads(data)


UnicodeDecodeError: ignored

In [ ]:
# 기본 지도 시각화 (TEST)

m = folium.Map(location=[37.541 , 126.986],zoom_start=12)

m

In [ ]:
# 지도 에 대한 다양한 옵션 적용 (TEST)

# 지도 형태 변경 , tiles 에 따라서 형태가 바뀌고 옵셥 여러개 있음. p.188 참조
m=folium.Map(location=[37.541 , 126.986],tiles='Stamen Toner' , zoom_start=12)

# 원하는 좌표에 반경 (radius) 표시 (남산)
folium.CircleMarker([37.5538 , 126.9810] ,radius= 50,
                    popup='Laurelhurst Park', color="#3246cc",
                    fill_color = '#3246cc').add_to(m)

# 원하는 좌표에 포인트 표시 (남산)
folium.Marker([37.5538 , 126.9810],popup = 'The Waterfront').add_to(m)

m

# 군집 마커 시각화

In [ ]:
# 서울 지도에 스타벅스 지점수 시각화

m=folium.Map(location=[37.541 , 126.986], zoom_start=12 , width='%100' , height='%100')

locations = list(zip(df.latitude , df.longitude))
cluster = plugins.MarkerCluster(locations=locations , popups=df['name'].tolist())
# MarkerCluster : 구역에 존재하는 스타벅스 지점의 수를 수자로 표현 (인터랙티브)

m.add_child(cluster)
m

# 도트맵 시각화


In [ ]:
# 서울 지도에 스타벅스 지점수 도트맵 시각화
 
m=folium.Map(location=[37.541 , 126.986], zoom_start=12 , width='%100' , height='%100')

locations = list(zip(df.latitude,df.longitude))

for i in range(len(locations)):
  folium.CircleMarker(location=locations[i],radius=1).add_to(m)

m

# 버블맵 시각화

In [ ]:
# 버블맵 시각화를 위한 데이터 전처리
# agg
df_m = df.groupby('gu_name').agg({'latitude':'mean','longitude':'mean','name':'count'}).reset_index()

df_m.head()

,gu_name,latitude,longitude,name
0,강남구,37.507603,127.044611,80
1,강동구,37.539914,127.137106,14
2,강북구,37.626866,127.026372,5
3,강서구,37.555716,126.841528,16
4,관악구,37.481759,126.944286,11


In [ ]:
# 버블맵 시각화

# 기본 지도 생성
m=folium.Map(location=[37.541 , 126.986], zoom_start=12 , tiles='Cartodb Positron', width='%100' , height='%100')

# 구별, 구분선 , 색상 설정
folium.Choropleth(geo_data=geo ,fill_color = 'gray').add_to(m)

# 버블맵 삽입
locations=list(zip(df.latitude , df.longitude))
for i in range(len(locations)):
  row = df_m.iloc[i]
  folium.CircleMarker(location=locations , radius = float(row.name/2), #버블크기 설정
                      fill_color='blue').add_to(m)

m


## JSON 파일 UTF-8 문제 있음.. 나중에 다시 시도 해보기로 함...

UnicodeDecodeError: ignored

## 위 실습에서 kaggle 의 json 파일에 오류가 있음, 다시 다운로드 해도 동일한 현상. 

- 다음 실습을 통해서 다시 시도 예정... 다른 파일로 성공하면 , 파일 문제 아니면 ,  colab 에서 쥬피터로 갈아타기. 버블 맵에서 문제 발생.

In [10]:
# 미국 실업률 정보의 코로플레스맵 시각화를 위한 데이터, json 불러오기
# https://www.kaggle.com/datasets/sewonghwang/us-unemployment

df2=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/study_dataset/us_/us_states_unemployment.csv',low_memory=False)

# 경계데이터 json 파일 불러오기

us_geo = '/content/drive/MyDrive/Colab Notebooks/study_dataset/us_/folium_us-states.json'

df2.head()

,State,Unemployment
0,AL,7.1
1,AK,6.8
2,AZ,8.1
3,AR,7.2
4,CA,10.1


# 코로플레스맵 시각화

In [12]:
# 미국 주별 실업률 코로플레스맵 시각화

# 미국 지도 시각화

m_en = folium.Map(loaction = [40,-98], zoom_start= 3 , tiles = 'Cartodb Positron')

# 지도에 주 경계선 , 실업률 데이터 연동

m_en.choropleth(geo_data = us_geo , 
                data = df2, 
                columns = ['State','Unemployment'],
                key_on = 'feature.id',
                fill_color = 'YlGn',
                legend_name = '실업률')
m_en


/usr/local/lib/python3.9/dist-packages/folium/folium.py:409: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


In [20]:
# 서울과 각국의 수도간의 커넥션 맵 시각화

# 서울과 도쿄 , 워싱턴 , 마닐라 , 파리 , 모스크바 위경도 입력

source_to_dest = zip([37.541,37.541,37.541,37.541,37.541], 
                     [35.6804, 38.9072, 14.5995, 48.8566,55.7558],
                     [126.986,126.986,126.986,126.986,126.986], 
                     [139.7690, -77.0369, 120.9842, 2.3522,37.6173])

fig = go.Figure()

## for 문으로 위 경도 입력 .

for a, b, c, d in source_to_dest:
  fig.add_trace(go.Scattergeo(lat=[a,b],
                           lon =[c,d],
                           mode = 'lines',
                           line = dict(width = 1 , color='red'),
                           opacity = 0.5 # 선투명도
                           ))
  
fig.update_layout(margin={'t':0 , "b":0 , "l":0 , "r":0 , "pad" :0},
                  showlegend=True,
                  geo = dict(showcountries=True) # 국가 경계선
                  )

fig.show()
